In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install torch transformers datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Load the XLNet tokenizer and model
model_name = 'xlnet-base-cased'  # You can also use 'xlnet-large-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
ds = load_dataset("ucberkeley-dlab/measuring-hate-speech")

Generating train split:   0%|          | 0/135556 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(ds['train'].select_columns(['text', 'hate_speech_score']))
df.head()

,text,hate_speech_score
0,Yes indeed. She sort of reminds me of the elde...,-3.90
1,The trans women reading this tweet right now i...,-6.52
2,Question: These 4 broads who criticize America...,0.36
3,It is about time for all illegals to go back t...,0.26
4,For starters bend over the one in pink and kic...,1.54


In [ ]:
scaler = MinMaxScaler()
df[['normalized_hate_speech_score']] = scaler.fit_transform(df[['hate_speech_score']])

In [ ]:
df.head()

,text,hate_speech_score,normalized_hate_speech_score
0,Yes indeed. She sort of reminds me of the elde...,-3.90,0.303279
1,The trans women reading this tweet right now i...,-6.52,0.124317
2,Question: These 4 broads who criticize America...,0.36,0.594262
3,It is about time for all illegals to go back t...,0.26,0.587432
4,For starters bend over the one in pink and kic...,1.54,0.674863


In [ ]:
def processed_text(text):
    lower_text = text.lower()
    no_special_char = re.sub(r'[^\w\s]', '', lower_text)
    final_str = no_special_char.strip()
    return final_str
df['text'] = df['text'].apply(processed_text)

In [ ]:
df.drop('hate_speech_score',axis=1)

,text,normalized_hate_speech_score
0,yes indeed she sort of reminds me of the elder...,0.303279
1,the trans women reading this tweet right now i...,0.124317
2,question these 4 broads who criticize america ...,0.594262
3,it is about time for all illegals to go back t...,0.587432
4,for starters bend over the one in pink and kic...,0.674863
...,...,...
135551,عاجل سماحة السيد_عبدالملك_بدرالدين_الحوثي نصر...,0.236339
135552,millions of yemenis participated in mass ralli...,0.269126
135553,abeshinzo realdonaldtrump shinzoabe 独裁者は行きますこれ...,0.399590
135554,millions of yemenis participated in mass ralli...,0.269126


In [ ]:
df['label'] = df['normalized_hate_speech_score'].apply(lambda x: 1 if x > 0.55 else 0)

In [ ]:
df_copy = df.copy()
df.drop('normalized_hate_speech_score',axis=1,inplace=True)

In [ ]:
dataset = Dataset.from_pandas(df)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True,max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split the dataset into train and validation sets
split_dataset = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

Map:   0%|          | 0/135556 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
